In [1]:
import pipelining
from selenium import webdriver
from bs4 import BeautifulSoup as Soup
import time
import lxml
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time
import pandas as pd
from torch.cuda import device
from tqdm import tqdm
from selenium.webdriver.chrome.options import Options
import requests
import os

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.distributed.pipelining import pipeline
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder
from transformers import BertTokenizer
from transformers import BertModel
from sklearn.model_selection import train_test_split
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

### Login + Page

In [8]:
# 抓取貼文內容--------------------
browser = webdriver.Chrome()
# 想要抓取資料的頁面網址
url = 'https://www.instagram.com/' 

# 前往頁面
browser.get(url)
# ------ 填入帳號與密碼 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.NAME, 'username')))

# ------ 網頁元素定位 ------
username_input = browser.find_element(By.NAME, "username")
password_input = browser.find_element(By.NAME, "password")
print("inputing username and password...")

# ------ 輸入帳號密碼 ------
username_input.send_keys("14han_15felix_")
password_input.send_keys("Yaya0329@Instagram")

# ------ 登入 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH,
'//*[@id="loginForm"]/div/div[3]/button/div')))
# ------ 網頁元素定位 ------
login_click = browser.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div')
# ------ 點擊登入鍵 ------
login_click.click()
time.sleep(20)
print("login successfully")

# 想要抓取資料的頁面網址
url = 'https://www.instagram.com' 
subUrlList = ['wendys', 'sonicdrivein', 'mcdonalds', 'mcdonalds_switzerland','mcdonaldscanada']

for subUrl in subUrlList:
    # 前往頁面
    browser.get(url + '/' + subUrl)
    time.sleep(2)
    # 往下滑並取得新的貼文連結
    post_all = []
    postList = []

    # Get scroll height.
    last_height = browser.execute_script("return document.body.scrollHeight")
    with tqdm(total=100) as pbar:
        while True:
            # Scroll down to the bottom.
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Wait to load the page.
            time.sleep(5)
    
            soup = Soup(browser.page_source, 'lxml')
            # 尋找所有的貼文連結
        
            for rowPost in soup.select('div._ac7v'):
                for elem in rowPost.select('div a'):
                    if elem['href'] in postList:
                        continue
                    postList.append(elem['href'])
                    img_elem = elem.select('div div img')
                    temp = []
                    temp.append(elem['href'])
                    try: 
                        temp.append(img_elem[0]['alt'])
                    except:
                        temp.append("")
                    temp.append(img_elem[0]['src'])
                    post_all.append(temp)
                    
            # Calculate new scroll height and compare with last scroll height.
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            pbar.update(1)
        
    print(subUrl+" 總共 " + str(len(post_all)) + " 篇貼文")
    
    # 存成csv
    postDataframe= pd.DataFrame(post_all)
    postDataframe.rename(columns = {0:'url', 1:'imgAlt', 2:'imgSrc'}, inplace = True)
    postDataframe.to_csv('Home_'+subUrl+'.csv', index = False)

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


inputing username and password...


KeyboardInterrupt: 

In [2]:
# 抓取貼文內容--------------------
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--window-size=334,778")
browser = webdriver.Chrome(options=chrome_options)
# 想要抓取資料的頁面網址
url = 'https://www.instagram.com/' 

# 前往頁面
browser.get(url)
# ------ 填入帳號與密碼 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.NAME, 'username')))

# ------ 網頁元素定位 ------
username_input = browser.find_element(By.NAME, "username")
password_input = browser.find_element(By.NAME, "password")
print("inputing username and password...")

# ------ 輸入帳號密碼 ------
username_input.send_keys("_blood_mango_")
password_input.send_keys("wangzeng1")

# ------ 登入 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH,
'//*[@id="loginForm"]/div/div[3]/button/div')))
# ------ 網頁元素定位 ------
login_click = browser.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div')
# ------ 點擊登入鍵 ------
login_click.click()
time.sleep(15)
print("login successfully")

# 想要抓取資料的頁面網址
url = 'https://www.instagram.com' 
subUrlList = ['sonicdrivein']
# 'wendys', 'mcdonalds', 'mcdonalds_switzerland', 'mcdonaldscanada', 'mcdonalds_switzerland'
# ,'sonicdrivein'

for subUrl in subUrlList:
    print("parsing "+subUrl+".csv")
    # insHome = pd.read_csv('Home_'+subUrl+'.csv')
    insHome = pd.read_csv('Done_'+subUrl+'.csv')
    print("parsing "+subUrl+".csv done")
    with tqdm(total=insHome.shape[0]) as pbar:
        counter = 0
        for index, row in insHome.iterrows(): #< 922
            if '/p/' in row['url'] and counter > 406 and counter < 923:
                browser.get(url + row['url'])
                time.sleep(10)
                html = browser.page_source
                soup = Soup(html, 'lxml')
                img_elem = soup.select('img.x5yr21d')
                text_elem = soup.select('h1._ap3a')

                insHome.at[index, 'imgSrc'] = img_elem[1]['src']
                try:
                    insHome.at[index, 'imgAlt'] = text_elem[0].text
                except:
                    insHome.at[index, 'imgAlt'] = ""
                insHome.to_csv('Done_'+subUrl+'.csv', index = False)
            pbar.update(1)
            counter = counter+1
            
    insHome['imgName'] = subUrl+'_'+insHome.index.astype(str)
    insHome.to_csv('Done_'+subUrl+'.csv', index = False)

inputing username and password...
login successfully
parsing mcdonalds_switzerland.csv
parsing mcdonalds_switzerland.csv done


100%|██████████| 2087/2087 [1:20:06<00:00,  2.30s/it] 


### Download Image

In [ ]:
subUrlList = ['wendys','mcdonalds', 'mcdonalds_switzerland','mcdonaldscanada','sonicdrivein']

for subUrl in subUrlList:
    print("parsing "+subUrl+".csv")
    insDone = pd.read_csv('Done_'+subUrl+'.csv')
    print("parsing "+subUrl+".csv done")
    with tqdm(total=insDone.shape[0]) as pbar:
        for index, row in insDone.iterrows():
            #create img folder
            if not os.path.exists(subUrl+'_img'):
                os.makedirs(subUrl+'_img')
            with open(subUrl+'_img/'+row['imgName']+'.jpg', 'wb') as f:
                try :
                    f.write(requests.get(row['imgSrc']).content)
                except:
                    print('Failed to download' + row['imgName']+'.jpg')
            pbar.update(1)

parsing wendys.csv
parsing wendys.csv done


100%|██████████| 371/371 [00:06<00:00, 54.63it/s]


parsing mcdonalds.csv
parsing mcdonalds.csv done


100%|██████████| 307/307 [00:10<00:00, 28.52it/s]


parsing mcdonalds_switzerland.csv
parsing mcdonalds_switzerland.csv done


...

100%|██████████| 850/850 [00:15<00:00, 53.73it/s]


parsing sonicdrivein.csv
parsing sonicdrivein.csv done


 23%|██▎       | 514/2240 [01:42<06:35,  4.37it/s]

Failed to downloadsonicdrivein_513.jpg


 78%|███████▊  | 1757/2240 [06:08<03:22,  2.38it/s]

In [33]:
# 抓取貼文內容--------------------
browser = webdriver.Chrome()
# 想要抓取資料的頁面網址
url = 'https://www.instagram.com/' 

# 前往頁面
browser.get(url)
# ------ 填入帳號與密碼 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.NAME, 'username')))

# ------ 網頁元素定位 ------
username_input = browser.find_element(By.NAME, "username")
password_input = browser.find_element(By.NAME, "password")
print("inputing username and password...")

# ------ 輸入帳號密碼 ------
username_input.send_keys("_wang_zeng")
password_input.send_keys("wangzeng1")

# ------ 登入 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH,
'//*[@id="loginForm"]/div/div[3]/button/div')))
# ------ 網頁元素定位 ------
login_click = browser.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div')
# ------ 點擊登入鍵 ------
login_click.click()
time.sleep(20)
print("login successfully")

# 想要抓取資料的頁面網址
url = 'https://www.instagram.com' 
subUrlList = ['wendys', 'sonicdrivein', 'mcdonalds', 'mcdonalds_switzerland','mcdonaldscanada']
post_count=[371, 2240, 307, 2087, 850]
counter = -1
for subUrl in subUrlList:
    download_counter = 0
    counter = counter+1
    print("parsing "+subUrl+".csv")
    insDone = pd.read_csv('Done_'+subUrl+'.csv')
    print("parsing "+subUrl+".csv done")
    
    # 前往頁面
    browser.get(url + '/' + subUrl)
    time.sleep(2)
    # 往下滑並取得新的貼文連結
    post_all = []
    postList = []

    # Get scroll height.
    last_height = browser.execute_script("return document.body.scrollHeight")
    with tqdm(total=post_count[counter]) as pbar:
        while True:
            # Scroll down to the bottom.
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Wait to load the page.
            time.sleep(5)
    
            soup = Soup(browser.page_source, 'lxml')
            # 尋找所有的貼文連結
        
            for rowPost in soup.select('div._ac7v'):
                for elem in rowPost.select('div a'):
                    if elem['href'] in postList:
                        continue
                        
                    imgName = insDone[insDone['url'] == elem['href']]['imgName']
                    if imgName.shape[0] > 0:
                        img_elem = elem.select('div div img')
                        imgUrl = img_elem[0]['src']
                        if not os.path.exists(subUrl+'_img'):
                            os.makedirs(subUrl+'_img')
                        with open(subUrl+'_img/'+imgName.values[0]+'.jpg', 'wb') as f:
                            try :
                                f.write(requests.get(imgUrl).content)
                                download_counter = download_counter+1
                            except:
                                print('Failed to download' + imgName+'.jpg')
                        postList.append(elem['href'])
                    pbar.update(1)
            # Calculate new scroll height and compare with last scroll height.
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            
    print(subUrl+" 總共 " + str(download_counter) + " 篇圖片已下載")
    

inputing username and password...
login successfully
parsing wendys.csv
parsing wendys.csv done


377it [02:55,  2.15it/s]                         


wendys 總共 371 篇圖片已下載
parsing sonicdrivein.csv
parsing sonicdrivein.csv done


2301it [17:40,  2.17it/s]                          


sonicdrivein 總共 2219 篇圖片已下載
parsing mcdonalds.csv
parsing mcdonalds.csv done


...

mcdonalds 總共 307 篇圖片已下載
parsing mcdonalds_switzerland.csv
parsing mcdonalds_switzerland.csv done


2202it [16:28,  2.23it/s]                          


mcdonalds_switzerland 總共 2050 篇圖片已下載
parsing mcdonaldscanada.csv
parsing mcdonaldscanada.csv done


877it [06:41,  2.18it/s]                         

mcdonaldscanada 總共 844 篇圖片已下載


In [4]:
# 抓取貼文內容--------------------
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--window-size=334,778")
browser = webdriver.Chrome(options=chrome_options)
# 想要抓取資料的頁面網址
url = 'https://www.instagram.com/' 

# 前往頁面
browser.get(url)
# ------ 填入帳號與密碼 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.NAME, 'username')))

# ------ 網頁元素定位 ------
username_input = browser.find_element(By.NAME, "username")
password_input = browser.find_element(By.NAME, "password")
print("inputing username and password...")

# ------ 輸入帳號密碼 ------
username_input.send_keys("_blood_mango_")
password_input.send_keys("wangzeng1")

# ------ 登入 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH,
'//*[@id="loginForm"]/div/div[3]/button/div')))
# ------ 網頁元素定位 ------
login_click = browser.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div')
# ------ 點擊登入鍵 ------
login_click.click()
time.sleep(15)
print("login successfully")

# 想要抓取資料的頁面網址
url = 'https://www.instagram.com' 


for subUrl in subUrlList:
    print("parsing "+subUrl+".csv")
    # insHome = pd.read_csv('Home_'+subUrl+'.csv')
    insHome = pd.read_csv('Done_'+subUrl+'.csv')
    print("parsing "+subUrl+".csv done")
    with tqdm(total=insHome.shape[0]) as pbar:
        counter = 0
        for index, row in insHome.iterrows(): #< 922
            if '/p/' in row['url'] and counter > 406 and counter < 923:
                browser.get(url + row['url'])
                time.sleep(10)
                html = browser.page_source
                soup = Soup(html, 'lxml')
                img_elem = soup.select('img.x5yr21d')
                text_elem = soup.select('h1._ap3a')

                insHome.at[index, 'imgSrc'] = img_elem[1]['src']
                
                    if '/reel/' in elem['href'] and imgName.shape[0] > 0:
                        img_elem = elem.select('div div img')
                        imgUrl = img_elem[0]['src']
                        if not os.path.exists(subUrl+'_img'):
                            os.makedirs(subUrl+'_img')
                        with open(subUrl+'_img/'+imgName.values[0]+'.jpg', 'wb') as f:
                            try :
                                f.write(requests.get(imgUrl).content)
                                download_counter = download_counter+1
                            except:
                                print('Failed to download' + imgName+'.jpg')
                        postList.append(elem['href'])
                    pbar.update(1)
            pbar.update(1)
            counter = counter+1
            
    insHome['imgName'] = subUrl+'_'+insHome.index.astype(str)
    insHome.to_csv('Done_'+subUrl+'.csv', index = False)

parsing sonicdrivein.csv
parsing sonicdrivein.csv done


100%|██████████| 2240/2240 [14:36<00:00,  2.56it/s] 


### Data Preprocessing

In [2]:
print(torch.__version__)
print(torch.cuda.is_available())

print(device)
torch.cuda.current_device()

2.4.1+cu124
True
cuda


0

In [3]:
torch.cuda.current_device()

0

In [20]:
pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-emoji", device=0)
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emoji")
test_token = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emoji")
# 将标题字段转换为tokens
def tokenize_bert(words):
    # 使用BERT Tokenizer对标题进行tokenize
    # tokens = tokenizer(words, truncation=True, padding='max_length', return_tensors='pt')
    tokens = test_token(words, truncation=True, padding='max_length', return_tensors='pt')
    return tokens['input_ids'].tolist()[0]

In [5]:
text = "👻🔥😂😁👍🤦‍♀️🤦‍♂️🤷‍♀️🤷‍♂️✌🤞😉😎🎶😢💖😜🐱‍🏍🐱‍👤🤳🎂🎉🌹💋👏🐱‍💻🐱‍🐉🐱‍👓✔👀😃✨😆🤔🤢🎁🫢 ha ha"
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))
print("=====================================")
print(test_token.encode(text))
print(test_token.decode(test_token.encode(text)))

[0, 31193, 2023, 6569, 10674, 8210, 18636, 9264, 18636, 10172, 31193, 8384, 6569, 10470, 18164, 17, 8384, 38718, 7471, 12605, 6569, 10470, 18164, 17, 8384, 38718, 9264, 12605, 6569, 10470, 18400, 17, 8384, 38718, 7471, 12605, 6569, 10470, 18400, 17, 8384, 38718, 9264, 12605, 39817, 14285, 6569, 10470, 17772, 18636, 23171, 18636, 12736, 6569, 12736, 19002, 18636, 7258, 6569, 10659, 25448, 18636, 48, 6569, 16948, 15389, 17, 8384, 6569, 9357, 8384, 6569, 16948, 15389, 17, 8384, 31193, 10470, 6569, 10470, 15264, 6569, 12736, 9264, 6569, 12736, 23171, 6569, 14285, 9253, 6569, 10659, 13859, 31193, 9357, 6569, 16948, 15389, 17, 8384, 6569, 10659, 2023, 6569, 16948, 15389, 17, 8384, 6569, 16948, 23171, 6569, 16948, 15389, 17, 8384, 31193, 9085, 39817, 10674, 31193, 7471, 18636, 862, 39817, 11423, 18636, 27819, 6569, 10470, 10674, 6569, 10470, 7258, 6569, 12736, 10172, 6569, 4958, 7258, 2489, 2489, 2]
<s>👻🔥😂😁👍🤦‍♀️🤦‍♂️🤷‍♀️🤷‍♂️✌🤞😉😎🎶😢💖😜🐱‍🏍🐱‍👤🤳🎂🎉🌹💋👏🐱‍💻🐱‍🐉🐱‍👓✔👀😃✨😆🤔🤢🎁🫢 ha ha</s>
[101, 100, 5292, 5292

In [21]:
print(len(tokenize_bert(text)))
print(len(tokenize_bert("text")))

512
512


In [19]:
print(len(tokenize_bert(text)))
print(len(tokenize_bert("text")))

143
3


In [14]:
from swin_transformer_v2 import SwinTransformerV2
from swin_transformer_v2 import swin_transformer_v2_t
from PIL import Image
from torchvision import transforms
# 載入預訓練的 SwinTransformerV2 模型
SwinTransformerV2 = swin_transformer_v2_t(in_channels=3,
                                          window_size=8,
                                          input_resolution=(256, 256),
                                          sequential_self_attention=False,
                                          use_checkpoint=False)

class SwinFeatures(nn.Module):
    def __init__(self):
        super(SwinFeatures, self).__init__()
        # 取出 ResNet-50 的前面部分，不包含最後的全連接層
        # self.features = nn.Sequential(*list(model.children())[:-1])
        self.features = nn.Sequential(*list(SwinTransformerV2.children())[:-1])
        
    def forward(self, x):
        x = self.features(x)
        return x


# 創建一個新的模型實例
feature_extractor = SwinFeatures()
feature_extractor.eval()  # 設置為評估模式

# 定義提取特徵的方法
def extract_features(image_path):
    # 加載並預處理圖像
    image = Image.open(image_path)
    # 確保圖像是RGB格式的
    image = image.convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = preprocess(image)
    image = image.unsqueeze(0)  # 添加批次維度

    # 通過新的模型提取特徵
    with torch.no_grad():
        features = feature_extractor(image)

    return features

# 以示例圖像路徑調用提取特徵的函數
image_path = './wendys_img/wendys_0.jpg'
features = extract_features(image_path)

# 檢視提取的特徵的形狀
print("Extracted features shape:", features.shape)

Extracted features shape: torch.Size([1, 96, 64, 64])


In [43]:
insFilter = pd.read_csv('./Original_File/Filter_mcdonalds_switzerland.csv')
#drop if isMeme is -2
print(insFilter.shape)
insFilter = insFilter[insFilter['isMeme'] != -2]
print(insFilter.shape)
insFilter.to_csv('Filter_mcdonalds_switzerland.csv', index = False)

(2087, 3)
(1129, 3)


In [46]:
subUrlList = ['wendys','mcdonalds', 'mcdonalds_switzerland','mcdonaldscanada','sonicdrivein']

for subUrl in subUrlList:
    print("parsing "+subUrl+".csv")
    insFilter = pd.read_csv('./Original_File/Filter_'+subUrl+'.csv')
    print("parsing "+subUrl+".csv done")
    
    insFilter['isMeme'] = insFilter['isMeme'].apply(lambda x: 1000 if x == 1 else(100 if x == 0 else 0))
    insFilter.to_csv('Filter_'+subUrl+'.csv', index = False)

parsing wendys.csv
parsing wendys.csv done
parsing mcdonalds.csv
parsing mcdonalds.csv done
parsing mcdonalds_switzerland.csv
parsing mcdonalds_switzerland.csv done
parsing mcdonaldscanada.csv
parsing mcdonaldscanada.csv done
parsing sonicdrivein.csv
parsing sonicdrivein.csv done


insFilter